<a href="https://colab.research.google.com/github/GiulioAppetito/Progetto_MOBD_2122/blob/main/test_routine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My\ Drive/ProgettoMOBD

In [19]:
import sklearn
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

np.random.seed = 123456

In [ ]:
# LETTURA TRAINING SET
train = pd.read_csv("train.csv", sep=",")
print(train.shape)

# QUI LETTURA DEL SUO TEST SET
test = pd.read_csv("test_set.csv", sep=",")
print(test.shape)

In [ ]:
# GESTIONE VALORI MANCANTI

x_test = test.iloc[:, :-1].to_numpy()
y_test = test.iloc[:, -1].to_numpy()

x_train = train.iloc[:, :-1].to_numpy()
y_train = train.iloc[:, -1].to_numpy()

#coerente con le operazioni fatte nella routine di addestramento
imp = SimpleImputer(missing_values=np.nan, strategy = 'most_frequent')
x_tr_notnan = imp.fit_transform(x_train)
# la sostituzione sul test set avviene con il most_frequent calcolato sul training set
x_ts_notnan = imp.transform(x_test)

print(x_ts_notnan.shape)

In [12]:
# GESTIONE FEATURES CATEGORICHE

def one_hot_encoding(df):
  final_df = pd.concat([df,pd.get_dummies(df['F1'])],axis = 1)
  final_df.drop('F1', axis=1, inplace=True)
  final_df = pd.concat([final_df,pd.get_dummies(df['F3'])],axis = 1)
  final_df.drop('F3', axis=1, inplace=True)
  final_df = pd.concat([final_df,pd.get_dummies(df['F7'])],axis = 1)
  final_df.drop('F7', axis=1, inplace=True)
  final_df = pd.concat([final_df,pd.get_dummies(df['F9'])],axis = 1)
  final_df.drop('F9', axis=1, inplace=True)
  final_df = pd.concat([final_df,pd.get_dummies(df['F8'])],axis = 1)
  final_df.drop('F8', axis=1, inplace=True)
  final_df = pd.concat([final_df,pd.get_dummies(df['F5'])],axis = 1)
  final_df.drop('F5', axis=1, inplace=True)
  final_df = pd.concat([final_df,pd.get_dummies(df['F6'])],axis = 1)
  final_df.drop('F6', axis=1, inplace=True)
  final_df = pd.concat([final_df,pd.get_dummies(df['F13'])],axis = 1)
  final_df.drop('F13', axis=1, inplace=True)
  return  final_df

In [ ]:
# PREPROCESSING
# coerente con quello eseguito nella fase di addestramento

x_test = pd.DataFrame(x_test)
x_test.columns=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13']
final_df_ts = one_hot_encoding(x_test)

# la align la stiamo usando perchè con get_dummies purtroppo se qualche valore che compare in uno dei due set
# non è presente nell'altro, non verrà generata la colonna corrispondente, per cui non è detto che i due dataframe che si 
# vanno ad usare abbiano stesso insieme di colonne, potrebbe succedere la stessa cosa se si andasse ad usare un dataset 
# che non si conosce oppure in previsione del caso in cui in futuro ci siano valori di cui non si è tenuto conto

x_train = pd.DataFrame(x_train)
x_train.columns=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13']
final_df_tr = one_hot_encoding(x_train)

x_tr, x_ts = final_df_tr.align(final_df_ts, join="outer", axis=1,fill_value=0)
#print(x_tr.tail())
#print(x_ts.tail())

In [ ]:
# SCALING
print("Robust Scaler")
scaler = RobustScaler()
x_tr_scaled = scaler.fit_transform(x_tr)
x_ts_scaled = scaler.transform(x_ts)

print("Media distribuzione di partenza: {}".format(np.mean(x_ts, axis=0)))
print("Media distribuzione scalata: {}".format(np.mean(x_ts_scaled, axis=0)))

print("Deviazione standard distribuzione di partenza: {}".format(np.std(x_ts, axis=0)))
print("Deviazione standard distribuzione scalata: {}".format(np.std(x_ts_scaled, axis=0)))

In [ ]:
from joblib import load
classifier = load('classifier.joblib')

y_ts_pred = classifier.predict(x_ts_scaled)

print("Accuracy on test set {}\n".format(accuracy_score(y_test, y_ts_pred)))
print("F1 score on test set {}".format(f1_score(y_test, y_ts_pred)))
print(confusion_matrix(y_test, y_ts_pred))